see also https://www.youtube.com/watch?v=hbTJvjfX1fA

In [1]:
import findspark
findspark.init()

In [25]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

In [83]:
from itertools import chain

In [3]:
#from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("Telcom").setMaster("local[4]")
sc = SparkContext(conf=conf)

In [4]:
sc

<SparkContext master=local[4] appName=Telcom>

In [20]:

sqlContext = SQLContext(sc)

#df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('./data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = sqlContext.read.format('csv').options(header='true', inferschema='true').load('./data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [21]:
df.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [22]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [48]:
df.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in df.columns]).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

# Process with PySpark

In [69]:
df.groupby('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5174|
|  Yes| 1869|
+-----+-----+



* imbalanced groups

In [70]:
df.select('tenure','TotalCharges','MonthlyCharges').describe().show()

+-------+------------------+------------------+------------------+
|summary|            tenure|      TotalCharges|    MonthlyCharges|
+-------+------------------+------------------+------------------+
|  count|              7043|              7043|              7043|
|   mean| 32.37114865824223|2283.3004408418697| 64.76169246059922|
| stddev|24.559481023094442| 2266.771361883145|30.090047097678482|
|    min|                 0|                  |             18.25|
|    max|                72|             999.9|            118.75|
+-------+------------------+------------------+------------------+



In [80]:
df.groupby('churn').pivot('gender').count().show()

+-----+------+----+
|churn|Female|Male|
+-----+------+----+
|   No|  2549|2625|
|  Yes|   939| 930|
+-----+------+----+



In [81]:
df.groupby('SeniorCitizen').pivot('churn').count().show()

+-------------+----+----+
|SeniorCitizen|  No| Yes|
+-------------+----+----+
|            1| 666| 476|
|            0|4508|1393|
+-------------+----+----+



In [91]:
def make_normalized_pivot(df, group_col, pivot_col, verbose=1):
    
    # https://stackoverflow.com/questions/40805808/percentage-count-per-group-and-pivot-with-pyspark
    levels = [x for x in chain(*df.select("churn").distinct().collect())]
    if verbose: print(f'levels of "{pivot_col}": {levels}')
    
    pivoted = df.groupby(group_col).pivot(pivot_col, levels).count()
    row_count = sum(F.coalesce(F.col(x), F.lit(0)) for x in levels)
    #row_count = sum(coalesce(col(x), lit(0)) for x in levels)
    adjusted = [(F.col(x) / row_count).alias(x) for x in levels]
    return pivoted.select(F.col(group_col), *adjusted)
                    
make_normalized_pivot(df=df, group_col='SeniorCitizen', pivot_col='churn').show()

levels of "churn": ['No', 'Yes']
+-------------+------------------+-------------------+
|SeniorCitizen|                No|                Yes|
+-------------+------------------+-------------------+
|            1|0.5831873905429071| 0.4168126094570928|
|            0| 0.763938315539739|0.23606168446026096|
+-------------+------------------+-------------------+



In [92]:
df.stat.crosstab("SeniorCitizen","InternetService").show()

+-----------------------------+----+-----------+----+
|SeniorCitizen_InternetService| DSL|Fiber optic|  No|
+-----------------------------+----+-----------+----+
|                            1| 259|        831|  52|
|                            0|2162|       2265|1474|
+-----------------------------+----+-----------+----+



In [96]:
df.select('SeniorCitizen','InternetService').sample(False,.002).show()

+-------------+---------------+
|SeniorCitizen|InternetService|
+-------------+---------------+
|            0|             No|
|            1|            DSL|
|            1|    Fiber optic|
|            0|            DSL|
|            0|            DSL|
|            0|             No|
|            0|            DSL|
|            0|            DSL|
|            0|             No|
|            0|    Fiber optic|
|            1|    Fiber optic|
|            0|             No|
|            1|    Fiber optic|
|            1|    Fiber optic|
|            0|    Fiber optic|
|            1|    Fiber optic|
|            0|             No|
+-------------+---------------+



# Process as standard pandas df

In [62]:
pd_df = df.sample(False, .05, seed=1).toPandas()

In [63]:
pd_df.__len__()

347

# Process as SQL

In [66]:
tmp_table_name = "churn_analysis"
df.createOrReplaceTempView(tmp_table_name)

In [67]:
type(df)

pyspark.sql.dataframe.DataFrame

In [68]:
%sql
select * from churn_analysis

SyntaxError: invalid syntax (<ipython-input-68-a45a80323020>, line 2)